In [ ]:
import os
import shutil
import platform
import numpy as np
import flopy
import modflowapi


In [ ]:
if "linux" in platform.platform().lower():
    lib_path = os.path.join("bin","linux","libmf6.so")
    exe_path = os.path.join("bin","linux","mf6")
elif "darwin" in platform.platform().lower() or "macos" in platform.platform().lower() :
    lib_path = os.path.join("bin","mac","libmf6.so")
    exe_path = os.path.join("bin","mac","mf6")
else:
    lib_path = os.path.join("bin","win","libmf6.dll")
    exe_path = os.path.join("bin","win","mf6.exe")     



In [ ]:
org_dir = "monthly_model_files_org"
work_dir = "temp"
if os.path.exists(work_dir):
    shutil.rmtree(work_dir)
shutil.copytree(org_dir,work_dir)
shutil.copy2(lib_path,os.path.join(work_dir,os.path.split(lib_path)[-1]))
shutil.copy2(exe_path,os.path.join(work_dir,os.path.split(exe_path)[-1]))
lib_name = os.path.split(lib_path)[-1]
exe_name = os.path.split(exe_path)[-1]

In [ ]:
c_d = os.getcwd()
os.chdir(work_dir)
os.system("mf6")
os.chdir(c_d)

In [ ]:
api_dir1 = "api1"
if os.path.exists(api_dir1):
    shutil.rmtree(api_dir1)
shutil.copytree(org_dir,api_dir1)
shutil.copy2(lib_path,os.path.join(api_dir1,os.path.split(lib_path)[-1]))
shutil.copy2(exe_path,os.path.join(api_dir1,os.path.split(exe_path)[-1]))

In [ ]:
gwf = modflowapi.ModflowApi(os.path.join(api_dir1, lib_name), working_directory=api_dir1)
gwf.initialize()

In [ ]:
# get current sim time
ctime = gwf.get_current_time()
# get ending sim time
etime = gwf.get_end_time()
# max number of iterations
max_iter = gwf.get_value(gwf.get_var_address("MXITER", "SLN_1"))
# let's do it!
while ctime < etime:
    # the length of this sim time
    dt = gwf.get_time_step()
    # prep the current time step
    gwf.prepare_time_step(dt)
    kiter = 0
    # prep to solve
    gwf.prepare_solve(1)
    # the current one-based stress period number
    stress_period = gwf.get_value(gwf.get_var_address("KPER", "TDIS"))[0]
    time_step = gwf.get_value(gwf.get_var_address("KSTP", "TDIS"))[0]
    # solve until converged
    while kiter < max_iter:
        if gwf.solve(1):
            print("flow stress period {0} converged with {1} iters".format(stress_period, kiter))
            break

        kiter += 1
    try:
        gwf.finalize_solve(1)
    except:
        pass

    gwf.finalize_time_step()
    # update current sim time
    ctime = gwf.get_current_time()
gwf.finalize()

Now let's do something more exciting.  We can use the API interface to build in operational rules for the pumping wells.  If the sfr downstream flow for the terminal reach is too low, we need to pump less, if the flow is high, we can pump more.

In [ ]:
api_dir2 = "api2"
if os.path.exists(api_dir2):
    shutil.rmtree(api_dir2)
shutil.copytree(org_dir,api_dir2)
shutil.copy2(lib_path,os.path.join(api_dir2,os.path.split(lib_path)[-1]))
shutil.copy2(exe_path,os.path.join(api_dir2,os.path.split(exe_path)[-1]))

In [ ]:
gwf = modflowapi.ModflowApi(os.path.join(api_dir2, lib_name), working_directory=api_dir2)
gwf.initialize()

In [ ]:
# get current sim time
ctime = gwf.get_current_time()
# get ending sim time
etime = gwf.get_end_time()
# max number of iterations
max_iter = gwf.get_value(gwf.get_var_address("MXITER", "SLN_1"))
# let's do it!
while ctime < etime:
    # the length of this sim time
    dt = gwf.get_time_step()
    # prep the current time step
    gwf.prepare_time_step(dt)
    kiter = 0
    # prep to solve
    gwf.prepare_solve(1)
    # the current one-based stress period number
    stress_period = gwf.get_value(gwf.get_var_address("KPER", "TDIS"))[0]
    time_step = gwf.get_value(gwf.get_var_address("KSTP", "TDIS"))[0]
    # solve until converged
    while kiter < max_iter:
        
        # if we have solved at least once and the solution is converged
        if gwf.solve(1) and kiter > 1:
            # get the sfr dsflow rate for the last reach
            addr = ["DSFLOW", "FREYBERG6", "SFR_1"]
            wbaddr = gwf.get_var_address(*addr)
            sfr_dsflow = gwf.get_value(wbaddr)
            print("flow stress period {0} converged with {1} iters, final sfr:{2:5.2f}".\
                  format(stress_period, kiter,sfr_dsflow[-1]))
            break
        
        # get the sfr dsflow rate for the last reach
        addr = ["DSFLOW", "FREYBERG6", "SFR_1"]
        wbaddr = gwf.get_var_address(*addr)
        sfr_dsflow = gwf.get_value(wbaddr)
        
        #get the wel extraction rates
        "FREYBERG6/WEL-1                   BOUND"
        addr = ["BOUND", "FREYBERG6", "WEl-1"]
        wbaddr = gwf.get_var_address(*addr)
        wel_bound = gwf.get_value_ptr(wbaddr)
        org = np.sum(wel_bound)
        
        # if the flow is too low, turn down the wells
        if sfr_dsflow[-1] < 2500:
            wel_bound[:] *= 0.8
        #otherewise if there is extra flow, turn up the wells
        elif sfr_dsflow[-1] > 3000:
            wel_bound[:] *= 1.5
        
        print("  ---  iter:{0}, sfr:{1:5.2f} org wel:{2:5.2f} new wel:{3:5.2f}".\
              format(kiter,sfr_dsflow[-1],org,np.sum(wel_bound)))
        kiter += 1
    try:
        gwf.finalize_solve(1)
    except:
        pass
    
    
    gwf.finalize_time_step()
    # update current sim time
    ctime = gwf.get_current_time()
    